<a href="https://colab.research.google.com/github/Kidus-Bellete/Cardiovascular-Disease-Prediction-with-Advanced-Machine-Learning-and-Explainable-AI/blob/main/cardiovascular_XAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install shap
#!pip install imblearn

In [ ]:
# Basic data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Deep learning with TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Model evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    precision_recall_curve
)

# Explainable AI
import shap


# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/XAI/heart_failure_clinical_records_dataset-1-1.csv')
df.head()

In [ ]:
df.info()
print("the total number of missing value in each features:")
print(df.isnull().sum())
df_duplicated = df.duplicated().sum()
print(f"Number of duplicate rows: {df_duplicated}")
df = df.drop_duplicates()

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(14, 10))
correlation_matrix = df.corr()
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            mask=mask, linewidths=0.5, vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Heart Failure Features', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Correlation with Target Variable (DEATH_EVENT)
plt.figure(figsize=(8, 6))
death_correlation = correlation_matrix['DEATH_EVENT'].drop('DEATH_EVENT').sort_values(ascending=False)
sns.barplot(x=death_correlation.values, y=death_correlation.index, hue=death_correlation.index, palette='viridis', legend=False)
plt.title('Correlation of Features with DEATH_EVENT', fontsize=16)
plt.xlabel('Correlation Coefficient')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of Numerical Features by Death Event
numerical_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction',
                      'platelets', 'serum_creatinine', 'serum_sodium', 'time']

fig, axes = plt.subplots(6, 3, figsize=(10, 20))
axes = axes.flatten()

for i, feature in enumerate(numerical_features):
    if i < len(axes):
        sns.histplot(data=df, x=feature, hue='DEATH_EVENT', kde=True, bins=25,
                    palette={0: 'green', 1: 'red'}, alpha=0.6, ax=axes[i])
        axes[i].set_title(f'Distribution of {feature} by Death Event', fontsize=14)
        axes[i].set_xlabel(feature, fontsize=12)
        axes[i].set_ylabel('Count', fontsize=12)
        axes[i].legend(['Survived', 'Died'], title='Outcome')
        axes[i].grid(linestyle='--', alpha=0.7)

# Remove any unused subplot
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots for Key Features by Death Event
key_features = ['age', 'ejection_fraction', 'serum_creatinine', 'time', 'serum_sodium']

fig, axes = plt.subplots(3, 2, figsize=(12, 10))
axes = axes.flatten()

for i, feature in enumerate(key_features):
    if i < len(axes):
        sns.boxplot(x='DEATH_EVENT', y=feature, data=df,
                   hue='DEATH_EVENT', palette={0: 'green', 1: 'red'},
                   legend=False, ax=axes[i])
        axes[i].set_title(f'Distribution of {feature} by Death Event', fontsize=14)
        axes[i].set_xlabel('Death Event (0=No, 1=Yes)', fontsize=12)
        axes[i].set_ylabel(feature, fontsize=12)
        axes[i].grid(linestyle='--', alpha=0.7)

# Remove any unused subplot
for i in range(len(key_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Categorical Features Analysis
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

fig, axes = plt.subplots(3, 2, figsize=(12, 10))
axes = axes.flatten()

for i, feature in enumerate(categorical_features):
    if i < len(axes):
        # Create a cross-tabulation
        cross_tab = pd.crosstab(df[feature], df['DEATH_EVENT'])
        cross_tab_percentage = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

        # Plot stacked bar chart
        cross_tab_percentage.plot(kind='bar', stacked=True,
                                colormap='viridis', ax=axes[i], rot=0)
        axes[i].set_title(f'Death Rate by {feature}', fontsize=14)
        axes[i].set_xlabel(f'{feature} (0=No, 1=Yes)', fontsize=12)
        axes[i].set_ylabel('Percentage (%)', fontsize=12)
        axes[i].legend(['Survived', 'Died'], title='Outcome')
        axes[i].grid(linestyle='--', alpha=0.7)

        # Add percentage labels
        for j, p in enumerate(axes[i].patches):
            width, height = p.get_width(), p.get_height()
            x, y = p.get_xy()
            if height > 5:  # Only label if percentage is greater than 5%
                axes[i].text(x + width/2, y + height/2, f'{height:.1f}%',
                            ha='center', va='center')

# Remove any unused subplot
for i in range(len(categorical_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Pairplot of Most Correlated Features
most_correlated_features = ['time', 'serum_creatinine', 'ejection_fraction', 'age', 'DEATH_EVENT']
sns.pairplot(df[most_correlated_features], hue='DEATH_EVENT',
             palette={0: 'green', 1: 'red'}, corner=True)
plt.suptitle('Pairplot of Most Correlated Features', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Age vs Time with Death Event
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='age', y='time', hue='DEATH_EVENT',
               palette={0: 'green', 1: 'red'}, size='serum_creatinine',
               sizes=(20, 200), alpha=0.7)
plt.title('Age vs. Follow-up Time by Death Event', fontsize=16)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Follow-up Time (days)', fontsize=14)
plt.grid(linestyle='--', alpha=0.7)
plt.legend(title='Death Event', labels=['Survived', 'Died'])
plt.tight_layout()
plt.show()

In [ ]:
# List of all continuous variables
continuous_variables = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets',
                   'serum_creatinine', 'serum_sodium', 'time']
# Standared Scaler
scaler = StandardScaler()
df[continuous_variables] = scaler.fit_transform(df[continuous_variables])

#MnMax Scaler
# minmax_scaler = MinMaxScaler()
# df[continuous_variables] = minmax_scaler.fit_transform(df[continuous_variables])

#Robust Scaler
# robust_scaler = RobustScaler()
# df[continuous_variables] = robust_scaler.fit_transform(df[continuous_variables])

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
# Define features (x) and target (y)
x = df.iloc[:, :-1].values  # All columns except the last one (DEATH_EVENT)
y = df.iloc[:, -1].values   # The last column (DEATH_EVENT)

# Check class distribution before SMOTE
counter_before = Counter(y)
print(f'Class distribution before SMOTE: {counter_before}')

# Apply SMOTE to balance the classes in DEATH_EVENT
oversample = SMOTE()
x, y = oversample.fit_resample(x, y)

# Check class distribution after SMOTE
counter_after = Counter(y)
print(f'Class distribution after SMOTE: {counter_after}')

# Split the data into training (70%) , val(15%) and tset(15%)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.30, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.50, random_state=42)

# Print the shapes of the datasets
print(f"Training set shape: {x_train.shape}")
print(f"Validation set shape: {x_val.shape}")
print(f"Test set shape: {x_test.shape}")

In [ ]:
# ML Models (Logistic Regression, Random Forest)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, roc_auc_score, classification_report)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Standardize data for Logistic Regression
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

def evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    """
    Comprehensive evaluation function for ML models
    """
    # Train model
    model.fit(X_train, y_train)

    # Generate predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    # Calculate metrics
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_prob) if y_prob is not None else None
    }

    # Print results
    print(f"\n{'='*40}")
    print(f"{model_name} Evaluation Results")
    print('='*40)
    print(classification_report(y_test, y_pred, target_names=['No Death', 'Death']))

    for metric, value in metrics.items():
        if value is not None:
            print(f"{metric}: {value:.4f}")

    return metrics

# =============================================
# Logistic Regression
# =============================================
print("\n" + "="*60)
print("LOGISTIC REGRESSION MODEL")
print("="*60)

logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg_metrics = evaluate_model(
    logreg,
    "Logistic Regression",
    x_train_scaled, y_train,
    x_test_scaled, y_test
)

# =============================================
# Random Forest
# =============================================
print("\n" + "="*60)
print("RANDOM FOREST MODEL")
print("="*60)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_metrics = evaluate_model(
    rf_model,
    "Random Forest",
    x_train, y_train,  # Random Forest doesn't require scaling
    x_test, y_test
)

# Feature importance visualization
X = df.drop('DEATH_EVENT', axis=1)
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
plt.title('Top 15 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

# =============================================
# Model Comparison
# =============================================
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)

# Create comparison dataframe
comparison_data = {
    'Model': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [
        logreg_metrics['Accuracy'],
        rf_metrics['Accuracy']
    ],
    'Precision': [
        logreg_metrics['Precision'],
        rf_metrics['Precision']
    ],
    'Recall': [
        logreg_metrics['Recall'],
        rf_metrics['Recall']
    ],
    'F1 Score': [
        logreg_metrics['F1'],
        rf_metrics['F1']
    ],
    'ROC AUC': [
        logreg_metrics['ROC AUC'],
        rf_metrics['ROC AUC']
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\nPerformance Comparison:")
print(comparison_df.round(4).to_string(index=False))

In [ ]:
# Neural Network Notebook

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Main Network Architecture
def build_simple_nn(input_dim, architecture='medium'):
    model = Sequential()

    if architecture == 'small':
        # Small architecture
        model.add(Dense(32, activation='relu', input_shape=(input_dim,)))
        model.add(Dropout(0.3))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))

    elif architecture == 'medium':
        # Medium architecture
        model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(32, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))

    elif architecture == 'large':
        # Larger architecture
        model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))

    # Compile model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy',
                         tf.keras.metrics.AUC(name='auc'),
                         tf.keras.metrics.Precision(name='precision'),
                         tf.keras.metrics.Recall(name='recall')])
    return model

def build_cnn_model(input_dim):
    """
    Build a 1D CNN model for tabular data
    """
    model = Sequential()

    # Reshape input for 1D CNN (samples, features, 1)
    model.add(tf.keras.layers.Reshape((input_dim, 1), input_shape=(input_dim,)))

    # First Conv1D block
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Dropout(0.5))

    # Second Conv1D block
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Dropout(0.5))

    # Third Conv1D block
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Flatten and dense layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy',
                         tf.keras.metrics.AUC(name='auc'),
                         tf.keras.metrics.Precision(name='precision'),
                         tf.keras.metrics.Recall(name='recall')])
    return model

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Train models with different architectures
architectures = ['small', 'medium', 'large', 'cnn']
models = {}
histories = {}
weighted_models = {}  # New dictionary for weighted models
weighted_histories = {}  # New dictionary for weighted histories

for arch in architectures:
    print(f"\n{'='*50}")
    print(f"Training {arch.upper()} {'CNN' if arch == 'cnn' else 'Neural Network'}")
    print(f"{'='*50}")

    # Build model
    if arch == 'cnn':
        model = build_cnn_model(x_train.shape[1])
    else:
        model = build_simple_nn(x_train.shape[1], arch)

    model.summary()

    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_auc', patience=20, mode='max', restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=10, min_lr=1e-6, verbose=1)
    ]

    # Train standard model
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=50,
        batch_size=16,
        callbacks=callbacks,
        verbose=1
    )

    models[arch] = model
    histories[arch] = history

    # Train weighted version (except for CNN)
    if arch != 'cnn':
        print(f"\nTraining WEIGHTED {arch.upper()} Neural Network")
        weighted_model = build_simple_nn(x_train.shape[1], arch)

        weighted_history = weighted_model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=50,
            batch_size=16,
            callbacks=callbacks,
            class_weight=class_weight_dict,
            verbose=1
        )

        weighted_models[f'{arch}_weighted'] = weighted_model
        weighted_histories[f'{arch}_weighted'] = weighted_history

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (roc_curve, roc_auc_score, classification_report,
                            confusion_matrix, precision_recall_curve, average_precision_score)
from sklearn.utils.class_weight import compute_class_weight

# =============================================
# 1. Evaluation Functions
# =============================================

def evaluate_ml_model(model, X, y, model_name, model_type="ML"):
    """
    Evaluate traditional ML models (Logistic Regression, SVM, Random Forest)
    """
    y_prob = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)

    # Metrics
    metrics = {
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred),
        'Recall': recall_score(y, y_pred),
        'F1': f1_score(y, y_pred),
        'ROC AUC': roc_auc_score(y, y_prob),
        'PR AUC': average_precision_score(y, y_prob)
    }

    # Print report
    print(f"\n{'='*60}")
    print(f"{model_name} ({model_type}) Evaluation")
    print('='*60)
    print(classification_report(y, y_pred, target_names=['No Death', 'Death']))

    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    return metrics, y_prob, y_pred

def evaluate_nn_model(model, X, y, model_name):
    """
    Evaluate neural network models
    """
    y_prob = model.predict(X, verbose=0).flatten()
    y_pred = (y_prob > 0.5).astype(int)

    # Metrics
    metrics = {
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred),
        'Recall': recall_score(y, y_pred),
        'F1': f1_score(y, y_pred),
        'ROC AUC': roc_auc_score(y, y_prob),
        'PR AUC': average_precision_score(y, y_prob)
    }

    # Print report
    print(f"\n{'='*60}")
    print(f"{model_name} (Neural Network) Evaluation")
    print('='*60)
    print(classification_report(y, y_pred, target_names=['No Death', 'Death']))

    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    return metrics, y_prob, y_pred

# =============================================
# 2. Visualization Functions
# =============================================

# def plot_confusion_matrix(y_true, y_pred, model_name):
#     cm = confusion_matrix(y_true, y_pred)
#     plt.figure(figsize=(6, 6))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['No Death', 'Death'],
#                 yticklabels=['No Death', 'Death'])
#     plt.title(f'{model_name} - Confusion Matrix')
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.show()

def plot_roc_curves(results_dict):
    plt.figure(figsize=(10, 8))
    for name, data in results_dict.items():
        fpr, tpr, _ = roc_curve(data['y_true'], data['y_prob'])
        auc_score = roc_auc_score(data['y_true'], data['y_prob'])
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc_score:.3f})', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_pr_curves(results_dict):
    plt.figure(figsize=(10, 8))
    for name, data in results_dict.items():
        precision, recall, _ = precision_recall_curve(data['y_true'], data['y_prob'])
        ap_score = average_precision_score(data['y_true'], data['y_prob'])
        plt.plot(recall, precision, label=f'{name} (AP = {ap_score:.3f})', linewidth=2)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curves')
    plt.legend(loc="upper right")
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_metric_comparison(comparison_df):
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC', 'PR AUC']
    plt.figure(figsize=(15, 10))
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 3, i)
        sns.barplot(data=comparison_df, x=metric, y='Model')
        plt.title(metric)
    plt.tight_layout()
    plt.show()

# =============================================
# 3. Main Evaluation Section
# =============================================

# Initialize storage for results
all_results = {}
comparison_data = []

# Evaluate ML Models (assuming they're trained in Notebook 1)
ml_models = {
    'Logistic Regression': logreg,
    'Random Forest': rf_model
}

for name, model in ml_models.items():
    metrics, y_prob, y_pred = evaluate_ml_model(model, x_test, y_test, name)
    all_results[name] = {'y_true': y_test, 'y_prob': y_prob, 'y_pred': y_pred}
    comparison_data.append({'Model': name, **metrics})

    # Plot confusion matrix
    #plot_confusion_matrix(y_test, y_pred, name)

# Evaluate Neural Networks (assuming they're trained in Notebook 2)
nn_models = {
    'Small NN': models['small'],
    'Medium NN': models['medium'],
    'Large NN': models['large'],
    'CNN': models['cnn']
}

for name, model in nn_models.items():
    metrics, y_prob, y_pred = evaluate_nn_model(model, x_test, y_test, name)
    all_results[name] = {'y_true': y_test, 'y_prob': y_prob, 'y_pred': y_pred}
    comparison_data.append({'Model': name, **metrics})

    # Plot confusion matrix
    #plot_confusion_matrix(y_test, y_pred, name)

# Evaluate Weighted Neural Networks (if available)
if 'weighted_models' in globals():
    weighted_nn_models = {
        'Small NN (Weighted)': weighted_models['small_weighted'],
        'Medium NN (Weighted)': weighted_models['medium_weighted'],
        'Large NN (Weighted)': weighted_models['large_weighted']
    }

    for name, model in weighted_nn_models.items():
        metrics, y_prob, y_pred = evaluate_nn_model(model, x_test, y_test, name)
        all_results[name] = {'y_true': y_test, 'y_prob': y_prob, 'y_pred': y_pred}
        comparison_data.append({'Model': name, **metrics})

        # Plot confusion matrix
        #plot_confusion_matrix(y_test, y_pred, name)

# =============================================
# 4. Comprehensive Comparison
# =============================================

# Create comparison dataframe
comparison_df = pd.DataFrame(comparison_data)

# Display comparison table
print("\n" + "="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(comparison_df.round(4).to_string(index=False))

# Visual comparisons
#plot_metric_comparison(comparison_df)
#plot_roc_curves(all_results)
#plot_pr_curves(all_results)

# =============================================
# 5. Neural Network Training Analysis
# =============================================

if 'histories' in globals():
    # Plot training histories for neural networks
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle('Neural Network Training Histories', fontsize=16)

    for i, (arch, history) in enumerate(histories.items()):
        # AUC plot
        axes[0, i].plot(history.history['auc'], label='Train AUC', alpha=0.8)
        axes[0, i].plot(history.history['val_auc'], label='Validation AUC', alpha=0.8)
        model_name = 'CNN' if arch == 'cnn' else f'{arch.capitalize()} NN'
        axes[0, i].set_title(f'{model_name} - AUC')
        axes[0, i].set_ylabel('AUC')
        axes[0, i].set_xlabel('Epoch')
        axes[0, i].legend()
        axes[0, i].grid(True, alpha=0.3)

        # Loss plot
        axes[1, i].plot(history.history['loss'], label='Train Loss', alpha=0.8)
        axes[1, i].plot(history.history['val_loss'], label='Validation Loss', alpha=0.8)
        axes[1, i].set_title(f'{model_name} - Loss')
        axes[1, i].set_ylabel('Loss')
        axes[1, i].set_xlabel('Epoch')
        axes[1, i].legend()
        axes[1, i].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
#!pip install lime

In [ ]:
# Notebook 4: Fixed XAI Analysis for All Models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
from sklearn.metrics import roc_auc_score
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("WORKING XAI ANALYSIS - ALL MODELS")
print("="*80)

# 1. INITIALIZATION AND DATA PREP
# =============================================

# Get feature names (handling both DataFrame and numpy array cases)
try:
    feature_names = x_train.columns.tolist()
except AttributeError:
    feature_names = [f'Feature {i}' for i in range(x_train.shape[1])]

# Prepare validation data (ensure it's numpy array)
if isinstance(x_val, pd.DataFrame):
    x_val_array = x_val.values
else:
    x_val_array = x_val.copy()

# Define all models to analyze
model_configs = {
    'Random Forest': {'model': rf_model, 'type': 'tree'},
    'Logistic Reg': {'model': logreg, 'type': 'linear'}
}

# For neural networks we'll use a different approach
nn_configs = {
    'Small NN': models['small'],
    'Medium NN': models['medium'],
    'Large NN': models['large'],
    'CNN': models['cnn']
}

# 2. PERMUTATION IMPORTANCE ANALYSIS
# =============================================

print("\nSTEP 1: RELIABLE PERMUTATION IMPORTANCE")
print("="*60)

def manual_permutation_importance(model, X, y, model_type, n_repeats=5):
    """Custom permutation importance calculation that works for all model types"""
    try:
        # Baseline score
        if model_type == 'nn':
            pred_fn = lambda x: model.predict(x, verbose=0).flatten()
        else:
            pred_fn = lambda x: model.predict_proba(x)[:, 1]

        baseline_score = roc_auc_score(y, pred_fn(X))

        # Calculate importance for each feature
        importances = np.zeros(X.shape[1])
        for i in range(X.shape[1]):
            scores = []
            for _ in range(n_repeats):
                X_permuted = X.copy()
                np.random.shuffle(X_permuted[:, i])
                permuted_score = roc_auc_score(y, pred_fn(X_permuted))
                scores.append(baseline_score - permuted_score)
            importances[i] = np.mean(scores)

        return importances

    except Exception as e:
        print(f"Error calculating importance: {str(e)}")
        return np.zeros(X.shape[1])

perm_results = {}
for name, config in tqdm(model_configs.items(), desc="Calculating Permutation Importance"):
    importances = manual_permutation_importance(
        config['model'],
        x_val_array,
        y_val,
        config['type']
    )
    perm_results[name] = importances

# For neural networks (slower but reliable)
print("\nCalculating for Neural Networks (this may take a while)...")
for name, model in tqdm(nn_configs.items(), desc="Neural Networks"):
    importances = manual_permutation_importance(
        model,
        x_val_array,
        y_val,
        'nn'
    )
    perm_results[name] = importances

# Create permutation importance dataframe
perm_df = pd.DataFrame(perm_results, index=feature_names)
perm_df['Average'] = perm_df.mean(axis=1)
perm_df = perm_df.sort_values('Average', ascending=False)

print("\nTop 10 Features by Average Permutation Importance:")
print(perm_df.head(10).round(4))

# 3. FEATURE IMPORTANCE VISUALIZATION (FIXED)
# =============================================

print("\nSTEP 2: FEATURE IMPORTANCE VISUALIZATION")
print("="*60)

# Fix 1: Heatmap of top features
plt.figure(figsize=(12, 8))
# Remove 'Average' column for heatmap
top_features = perm_df.head(15).drop('Average', axis=1)
sns.heatmap(top_features.T,
            annot=True, fmt=".3f",
            cmap="YlOrRd",
            cbar_kws={'label': 'Importance Score'})
plt.title("Top 15 Features - Importance Across Models", fontsize=14, pad=20)
plt.xlabel("Features", fontsize=12)
plt.ylabel("Models", fontsize=12)
plt.tight_layout()
plt.show()

# Fix 2: Line plot for top 5 features across models
top_5_features = perm_df.index[:5]
plt.figure(figsize=(12, 6))

# Get model names (excluding 'Average')
model_names = [col for col in perm_df.columns if col != 'Average']

for feature in top_5_features:
    # Get values for each model (excluding 'Average')
    values = [perm_df.loc[feature, model] for model in model_names]
    plt.plot(model_names, values, 'o-', label=feature, linewidth=2, markersize=6)

plt.title("Importance Scores for Top 5 Features Across Models", fontsize=14, pad=20)
plt.ylabel("Importance Score", fontsize=12)
plt.xlabel("Models", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Additional Visualization: Bar plot of average importances
plt.figure(figsize=(10, 6))
top_10_avg = perm_df.head(10)['Average']
bars = plt.bar(range(len(top_10_avg)), top_10_avg.values,
               color='skyblue', edgecolor='navy', alpha=0.7)
plt.title("Top 10 Features by Average Importance", fontsize=14, pad=20)
plt.xlabel("Features", fontsize=12)
plt.ylabel("Average Importance Score", fontsize=12)
plt.xticks(range(len(top_10_avg)), top_10_avg.index, rotation=45)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# 4. MODEL COMPARISON ANALYSIS
# =============================================

print("\nSTEP 3: MODEL COMPARISON ANALYSIS")
print("="*60)

# Create a comprehensive comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Feature importance correlation between models
corr_matrix = perm_df.drop('Average', axis=1).corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, ax=axes[0,0])
axes[0,0].set_title("Model Agreement on Feature Importance")

# Plot 2: Distribution of importance scores by model
model_data = []
for model in model_names:
    for importance in perm_df[model]:
        model_data.append({'Model': model, 'Importance': importance})
model_comparison_df = pd.DataFrame(model_data)

sns.boxplot(data=model_comparison_df, x='Model', y='Importance', ax=axes[0,1])
axes[0,1].set_title("Distribution of Importance Scores by Model")
axes[0,1].tick_params(axis='x', rotation=45)

# Plot 3: Top features by each model
top_by_model = {}
for model in model_names:
    top_by_model[model] = perm_df.nlargest(5, model).index.tolist()

# Create a stacked bar chart showing how often each feature appears in top 5
feature_counts = {}
for features in top_by_model.values():
    for feature in features:
        feature_counts[feature] = feature_counts.get(feature, 0) + 1

sorted_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)[:10]
features, counts = zip(*sorted_features)

axes[1,0].bar(range(len(features)), counts, color='lightcoral', alpha=0.7)
axes[1,0].set_title("Features Most Often in Top 5 (Across Models)")
axes[1,0].set_xlabel("Features")
axes[1,0].set_ylabel("Times in Top 5")
axes[1,0].set_xticks(range(len(features)))
axes[1,0].set_xticklabels(features, rotation=45)

# Plot 4: Stability vs Importance scatter
stability = perm_df.drop('Average', axis=1).std(axis=1)
importance = perm_df['Average']

scatter = axes[1,1].scatter(importance, stability, alpha=0.6, s=60)
axes[1,1].set_xlabel("Average Importance")
axes[1,1].set_ylabel("Importance Stability (Std Dev)")
axes[1,1].set_title("Feature Importance vs Stability")

# Annotate most important and most stable features
for i, txt in enumerate(perm_df.index[:5]):  # Top 5 by importance
    axes[1,1].annotate(txt, (importance.iloc[i], stability.iloc[i]),
                      xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.show()

# 5. KEY INSIGHTS AND RECOMMENDATIONS
# =============================================

print("\nSTEP 4: KEY INSIGHTS AND RECOMMENDATIONS")
print("="*60)

# Identify consensus important features
threshold = perm_df['Average'].quantile(0.75)
consensus_features = perm_df[perm_df['Average'] > threshold].index.tolist()

print("\n1. CONSENSUS IMPORTANT FEATURES (Across All Models):")
for i, feat in enumerate(consensus_features[:5], 1):
    print(f"  {i}. {feat} (Avg Importance: {perm_df.loc[feat, 'Average']:.3f})")

# Identify model-specific important features
print("\n2. MODEL-SPECIFIC IMPORTANT FEATURES:")
for model in model_names:
    top_feat = perm_df[model].idxmax()
    print(f"  - {model}: {top_feat} (Score: {perm_df.loc[top_feat, model]:.3f})")

# Stability analysis
print("\n3. FEATURE STABILITY ANALYSIS:")
stability_scores = perm_df.drop('Average', axis=1).std(axis=1)
stable_features = stability_scores.sort_values().index[:3]
volatile_features = stability_scores.sort_values(ascending=False).index[:3]

print("  Most Stable Features (consistent across models):")
for feat in stable_features:
    print(f"    {feat} (Std: {stability_scores[feat]:.3f})")

print("\n  Most Volatile Features (model-dependent importance):")
for feat in volatile_features:
    print(f"    {feat} (Std: {stability_scores[feat]:.3f})")

# Model agreement analysis
print("\n4. MODEL AGREEMENT ANALYSIS:")
correlation_scores = corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)]
print(f"  Average correlation between models: {np.mean(correlation_scores):.3f}")
print(f"  Models agree most: {corr_matrix.max().max():.3f}")
print(f"  Models agree least: {corr_matrix.min().min():.3f}")

# Final recommendations
print("\n5. RECOMMENDATIONS:")
print("  • Focus on consensus features for robust model interpretability")
print("  • Investigate model-specific important features for specialized insights")


print("\n" + "="*80)
print("ENHANCED XAI ANALYSIS COMPLETE")
print("="*80)

# Save comprehensive results
results_summary = {
    'feature_importance': perm_df,
    'model_correlations': corr_matrix,
    'stability_scores': stability_scores,
    'consensus_features': consensus_features,
    'top_by_model': top_by_model
}

# Save main results
perm_df.to_csv("feature_importance_results.csv")
corr_matrix.to_csv("model_correlation_matrix.csv")

print("\nResults saved:")
print("- feature_importance_results.csv")
print("- model_correlation_matrix.csv")